# _include_repo

In [ ]:
#|default_exp cmds._include_repo
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()

In [ ]:
#|top_export
from pathlib import Path

from repoyard.config import get_config
from repoyard import const

In [ ]:
#|set_func_signature
async def include_repo(
    config_path: Path,
    repo_index_name: str,
    soft_interruption_enabled: bool = True,
):
    """
    """
    ...

Set up testing args

In [ ]:
from tests.utils import *
remote_name, remote_rclone_path, config, config_path, data_path = create_repoyards()

In [ ]:
# Args
from repoyard.cmds import new_repo
config_path = config_path
repo_index_name = new_repo(config_path=config_path, repo_name="test_repo", storage_location="my_remote")
soft_interruption_enabled = True

# Function body

Process args

In [ ]:
#|export
config = get_config(config_path)

In [ ]:
from repoyard.cmds import sync_repo, exclude_repo
# Sync the repo
await sync_repo(config_path=config_path, repo_index_name=repo_index_name)
# Remove the repo from the local store to test the inclusion
await exclude_repo(config_path=config_path, repo_index_name=repo_index_name)

Check if repo is already included

In [ ]:
#|export
from repoyard._models import get_repoyard_meta
repoyard_meta = get_repoyard_meta(config)

if repo_index_name not in repoyard_meta.by_index_name:
    raise ValueError(f"Repo '{repo_index_name}' does not exist.")

repo_meta = repoyard_meta.by_index_name[repo_index_name]

if repo_meta.check_included(config):
    raise ValueError(f"Repo '{repo_index_name}' is already included.")

Include it

In [ ]:
#|export
from repoyard.cmds import sync_repo
from repoyard._models import RepoPart
from repoyard._utils.sync_helper import sync_helper, SyncSetting, SyncDirection

# First force sync the data
await sync_repo(
    config_path=config_path,
    repo_index_name=repo_index_name,
    sync_direction=SyncDirection.PULL,
    sync_setting=SyncSetting.FORCE,
    sync_choices=[RepoPart.DATA],
    soft_interruption_enabled=soft_interruption_enabled,
)

# Then sync the rest
await sync_repo(
    config_path=config_path,
    repo_index_name=repo_index_name,
    sync_direction=None,
    sync_setting=SyncSetting.CAREFUL,
    sync_choices=[RepoPart.META, RepoPart.CONF],
    soft_interruption_enabled=soft_interruption_enabled,
);

In [ ]:
# Should now be included
assert repo_meta.check_included(config)